<목차>
---
1. 텐서플로 기초 - 그래프 생성과 그래프 실행
2. Placeholder
3. 선형회귀 및 경사하강법 알고리즘
    - 머신러닝의 기본 프로세스
    - 선형 회귀 알고리즘 구현 및 변수
4. 텐서보드를 이용한 그래프 시각화

# 1. Tensorflow 기초 - 그래프 생성과 그래프 실행
---
rank 0 -> scalar  
rank 1 -> vector  
rank 2 -> array  
rank 3 over -> **tensor**
  
tensor는 **계산 그래프 구조**를 통해 이동  
(node에 연산, 변수, 상수 정의하고 edge를 통해 tensor를 주고받으면서 계산 수행)

In [ ]:
#pip install tensorflow
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

### 상수 텐서 선언

**tf.constant**(value, dtype=None, shape=None, name='Const')  
shape: 데이터 형태  
name: 텐서의 이름





In [ ]:
#그래프 생성
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) #기본값 float32
print(node1, node2) #노드 정보만 출력

#그래프 실행
sess = tf.Session()
print(sess.run([node1, node2]))

#그래프 생성+실행
node3 = tf.add(node1, node2)
print(node3)
print(sess.run(node3))

sess.close()

# 2. Placeholder
---
임의의 값을 받아서 임의의 값 출력  
**tf.placeholder**(dtype, shape=None, name=None)

In [6]:
#그래프 생성
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b

#그래프 실행
sess = tf.Session()
print(sess.run(adder_node, feed_dict={a:3, b:4.5}))
print(sess.run(adder_node, feed_dict={a:[1,3], b:[2,4]}))
#feed_dict 부분에 노드에 넣을 값 지정

#그래프 생성+실행
add_and_triple = adder_node * 3
print(sess.run(add_and_triple, feed_dict={a:3, b:4.5}))

sess.close()

# 3. 선형회귀 & 경사하강법 Algorithm
---
##  머신러닝의 기본 프로세스
1) 가설정의  
  - 변수 node를 선언해서 파라미터로 사용  
  - placeholder node를 선언해서 input, target 데이터를 받음

2) 손실함수(비용함수) 정의
  - 적절한 파라미터값을 알아내기 위함  
  - 파라미터가 최적화되었을 때 더 작은 값을 갖음  
   ex) 평균제곱오차(MSE)  

3) 최적화 정의  
  - 파라미터를 업데이트하는 알고리즘  
  ex) 경사하강법

## 선형회귀 알고리즘 구현 및 변수
**tf.Variable**(initial_value = None, trainable = True, name = None)  
파라미터를 선언할 수 있는 클래스  
initial_value: 변수의 shape를 포함한 초기값  

**초기화로 사용할 수 있는 연산들**
* tf.random_normal: 정규분포에서 임의의 값 추출  
* tf.truncated_normal: 끝이 잘린 정규분포에서 임의의 값 추출  
* tf.random_uniform: 균등 분포에서 임의의 값 추출   
* tf.constant: 특정한 상수값으로 지정한 행렬  
* tf.zero: 모두 0으로 지정한 행렬  
* tf.ones: 모두 1로 지정한 행렬

In [14]:
#선형 회귀 모델의 그래프 구조 정의
W = tf.Variable(tf.random.normal(shape = [1]))
b = tf.Variable(tf.random.normal(shape = [1]))
x = tf.placeholder(tf.float32)
linear_model = W*x + b

#타겟 데이터 정의
y = tf.placeholder(tf.float32)

#손실함수 정의 (MSE)
loss = tf.reduce_mean(tf.square(linear_model - y))

#최적화 정의 (경사하강법)
optimizer = tf.train.GradientDescentOptimizer(0.01)
train_step = optimizer.minimize(loss)

#데이터 준비
x_train = [1,2,3,4]
y_train = [2,4,6,8]

#값 초기화 
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#실행
for i in range(100):
    sess.run(train_step, feed_dict={x: x_train, y: y_train})
    
#학습 결과 확인
x_test= [3.5, 5, 5.5, 6]
print(sess.run(linear_model, feed_dict={x: x_test}))

sess.close()

[ 6.7908325  9.230441  10.043644  10.856846 ]


# 4. TensorBoard를 이용한 그래프 시각화
학습이 올바른 방향으로 진행되고 있는지 측정하기 위해  

- 방법1) tf.summary.scalar API를 이용
  - 스텝마다 손실 함수 값을 출력

In [19]:
#손실 함수 정의 
loss = tf.reduce_mean(tf.square(linear_model - y))

#요약 정보 정의
tf.summary.scalar('loss', loss)

#요약 정보를 하나로 합치기
merged = tf.summary.merge_all()

#폴더 경로 설정
tensorboard_writer = tf.summary.FileWriter('./tensorboard_log', sess.graph)

#경사하강법 1000번 수행
for i in range(1000):
  sess.run(train_step, feed_dict={x: x_train, y:y_train})
  #스텝마다 요약정보 저장
  summary = sess.run(merged, feed_dict={x:x_train, y:y_train})
  tensorboard_writer.add_summary(summary, i)

FailedPreconditionError: ignored

- 방법2) 계산 그래프 시각화
  - 손실 함수 값을 시각화

텐서보드로 이동  
tensorboard --logdir=./logs/fit/ 